# Random Forest

### Imports

In [1]:
! pip install graphviz
! pip install dl8.5
! pip install chefboost

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time as t
import random

! pip install graphviz

from IPython.display import SVG
from graphviz import Source

from sklearn import tree
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

### Helper Functions

In [3]:
def accuracy(y_actual, y_predicted):
    if(not len(y_actual) == len(y_predicted)):
        print("Lengths don't match")
        return 0
    correct = 0
    for i in range(len(y_actual)):
        if y_actual[i] == y_predicted[i]:
            correct += 1
    return 1.0 * correct / len(y_actual)

def clean_df(df):
    for (field, vals) in df.iteritems():
#         print("{:} | {:}".format(field, type(vals[0])))
        if type(vals[0]) == str:
            strings = []
            for s_ind in vals.index:
                if vals[s_ind] not in strings:
                    strings += [vals[s_ind]]
                df.at[s_ind, field] = strings.index(vals[s_ind])
#             print(strings)
    return df

def make_rfc(num_trees, max_depth):
    rfc = RandomForestClassifier()
    rfc.criterion = 'entropy'
    rfc.max_tree_depth = max_depth
    rfc.n_estimators = num_trees
    return rfc

### Data Importing and Processing

In [4]:
start_time = t.time()
df = pd.read_csv("../data/train.csv")
df_test = pd.read_csv("../data/test.csv")

# print(df)

### Data Processing

In [10]:
# Remove NaN Columns

df = df.dropna()
print(df_test.shape[0])
df_test = df_test.dropna()
print(df_test.shape[0])

# Make string values into numbers

clean_df(df)
clean_df(df_test)

# Not 10 like normal since we already have a test partition
num_folds = 9 

cv_length = 1.0 * df.shape[0] / num_folds

partitions = [df.iloc[int(i * cv_length):int((i+1) * cv_length),:] for i in range(num_folds)]
# print([len(partitions[i]) for i in range(len(partitions))])


2390
2390


### Model

In [6]:
# Rotate the folds to tune the parameters

results = []
max_tree_depths = range(1, 15)
num_trees = range(1,50)
num_coord_descs = 5
opt_num_trees = random.choice(num_trees)
opt_tree_depth = random.choice(max_tree_depths)

for c in range(num_coord_descs):
    print("Number descent:", c)
    
    results = []

    for i in range(num_folds):
        validation = partitions[i]
        training = pd.concat([j for j in partitions if not j.equals(validation)])

        y_train = training['Decision']
        X_train = training.loc[:, df.columns != 'Decision']

        y_valid = validation['Decision']
        X_valid = validation.loc[:, df.columns != 'Decision']

        results_for_fold = []

        for n in num_trees:
            rfc = make_rfc(n, opt_tree_depth)
            rfc.fit(X_train, y_train)
            a = accuracy(np.array(y_valid), rfc.predict(X_valid))
            results_for_fold += [a]
            print("Accuracy for {:4d} trees with depth {:2d} on fold {:2d} is {:.3f}".format(n, opt_tree_depth, i, a))

        results += [results_for_fold]
        
    results_for_n = np.array(results).T
    avg_acc = [np.mean(results_for_n[j]) for j in range(len(results_for_n))]
    
    opt_num_trees = num_trees[avg_acc.index(np.max(avg_acc))]
    print("Optimal number of trees:", opt_num_trees)
    
    results = []

    for i in range(num_folds):
        validation = partitions[i]
        training = pd.concat([j for j in partitions if not j.equals(validation)])

        y_train = training['Decision']
        X_train = training.loc[:, df.columns != 'Decision']

        y_valid = validation['Decision']
        X_valid = validation.loc[:, df.columns != 'Decision']

        results_for_fold = []

        for d in max_tree_depths:
            rfc = make_rfc(n, opt_tree_depth)
            rfc.fit(X_train, y_train)
            a = accuracy(np.array(y_valid), rfc.predict(X_valid))
            results_for_fold += [a]
            print("Accuracy for {:4d} trees with depth {:2d} on fold {:2d} is {:.3f}".format(opt_num_trees, d, i, a))

        results += [results_for_fold]
        
    results_for_d = np.array(results).T
    avg_acc = [np.mean(results_for_d[j]) for j in range(len(results_for_d))]
    
    opt_tree_depth = max_tree_depths[avg_acc.index(np.max(avg_acc))]
    print("Optimal tree depth:", opt_tree_depth)

Number descent: 0
Accuracy for    1 trees with depth  9 on fold  0 is 0.577
Accuracy for    2 trees with depth  9 on fold  0 is 0.580
Accuracy for    3 trees with depth  9 on fold  0 is 0.678
Accuracy for    4 trees with depth  9 on fold  0 is 0.660
Accuracy for    5 trees with depth  9 on fold  0 is 0.687
Accuracy for    6 trees with depth  9 on fold  0 is 0.686
Accuracy for    7 trees with depth  9 on fold  0 is 0.681
Accuracy for    8 trees with depth  9 on fold  0 is 0.680
Accuracy for    9 trees with depth  9 on fold  0 is 0.697
Accuracy for   10 trees with depth  9 on fold  0 is 0.684
Accuracy for   11 trees with depth  9 on fold  0 is 0.708
Accuracy for   12 trees with depth  9 on fold  0 is 0.709
Accuracy for   13 trees with depth  9 on fold  0 is 0.740
Accuracy for   14 trees with depth  9 on fold  0 is 0.713
Accuracy for   15 trees with depth  9 on fold  0 is 0.725
Accuracy for   16 trees with depth  9 on fold  0 is 0.741
Accuracy for   17 trees with depth  9 on fold  0 is 0.

Accuracy for   44 trees with depth  9 on fold  2 is 0.732
Accuracy for   45 trees with depth  9 on fold  2 is 0.740
Accuracy for   46 trees with depth  9 on fold  2 is 0.729
Accuracy for   47 trees with depth  9 on fold  2 is 0.733
Accuracy for   48 trees with depth  9 on fold  2 is 0.729
Accuracy for   49 trees with depth  9 on fold  2 is 0.730
Accuracy for    1 trees with depth  9 on fold  3 is 0.615
Accuracy for    2 trees with depth  9 on fold  3 is 0.597
Accuracy for    3 trees with depth  9 on fold  3 is 0.673
Accuracy for    4 trees with depth  9 on fold  3 is 0.655
Accuracy for    5 trees with depth  9 on fold  3 is 0.709
Accuracy for    6 trees with depth  9 on fold  3 is 0.690
Accuracy for    7 trees with depth  9 on fold  3 is 0.702
Accuracy for    8 trees with depth  9 on fold  3 is 0.718
Accuracy for    9 trees with depth  9 on fold  3 is 0.721
Accuracy for   10 trees with depth  9 on fold  3 is 0.712
Accuracy for   11 trees with depth  9 on fold  3 is 0.734
Accuracy for  

Accuracy for   39 trees with depth  9 on fold  5 is 0.747
Accuracy for   40 trees with depth  9 on fold  5 is 0.738
Accuracy for   41 trees with depth  9 on fold  5 is 0.748
Accuracy for   42 trees with depth  9 on fold  5 is 0.740
Accuracy for   43 trees with depth  9 on fold  5 is 0.742
Accuracy for   44 trees with depth  9 on fold  5 is 0.759
Accuracy for   45 trees with depth  9 on fold  5 is 0.756
Accuracy for   46 trees with depth  9 on fold  5 is 0.746
Accuracy for   47 trees with depth  9 on fold  5 is 0.734
Accuracy for   48 trees with depth  9 on fold  5 is 0.746
Accuracy for   49 trees with depth  9 on fold  5 is 0.752
Accuracy for    1 trees with depth  9 on fold  6 is 0.626
Accuracy for    2 trees with depth  9 on fold  6 is 0.592
Accuracy for    3 trees with depth  9 on fold  6 is 0.644
Accuracy for    4 trees with depth  9 on fold  6 is 0.640
Accuracy for    5 trees with depth  9 on fold  6 is 0.673
Accuracy for    6 trees with depth  9 on fold  6 is 0.695
Accuracy for  

Accuracy for   34 trees with depth  9 on fold  8 is 0.719
Accuracy for   35 trees with depth  9 on fold  8 is 0.737
Accuracy for   36 trees with depth  9 on fold  8 is 0.734
Accuracy for   37 trees with depth  9 on fold  8 is 0.721
Accuracy for   38 trees with depth  9 on fold  8 is 0.724
Accuracy for   39 trees with depth  9 on fold  8 is 0.748
Accuracy for   40 trees with depth  9 on fold  8 is 0.726
Accuracy for   41 trees with depth  9 on fold  8 is 0.740
Accuracy for   42 trees with depth  9 on fold  8 is 0.736
Accuracy for   43 trees with depth  9 on fold  8 is 0.741
Accuracy for   44 trees with depth  9 on fold  8 is 0.714
Accuracy for   45 trees with depth  9 on fold  8 is 0.730
Accuracy for   46 trees with depth  9 on fold  8 is 0.738
Accuracy for   47 trees with depth  9 on fold  8 is 0.733
Accuracy for   48 trees with depth  9 on fold  8 is 0.743
Accuracy for   49 trees with depth  9 on fold  8 is 0.734
Optimal number of trees: 39
Accuracy for   39 trees with depth  1 on fol

Accuracy for   39 trees with depth 14 on fold  8 is 0.744
Optimal tree depth: 11
Number descent: 1
Accuracy for    1 trees with depth 11 on fold  0 is 0.652
Accuracy for    2 trees with depth 11 on fold  0 is 0.572
Accuracy for    3 trees with depth 11 on fold  0 is 0.649
Accuracy for    4 trees with depth 11 on fold  0 is 0.618
Accuracy for    5 trees with depth 11 on fold  0 is 0.698
Accuracy for    6 trees with depth 11 on fold  0 is 0.680
Accuracy for    7 trees with depth 11 on fold  0 is 0.717
Accuracy for    8 trees with depth 11 on fold  0 is 0.700
Accuracy for    9 trees with depth 11 on fold  0 is 0.704
Accuracy for   10 trees with depth 11 on fold  0 is 0.704
Accuracy for   11 trees with depth 11 on fold  0 is 0.690
Accuracy for   12 trees with depth 11 on fold  0 is 0.708
Accuracy for   13 trees with depth 11 on fold  0 is 0.738
Accuracy for   14 trees with depth 11 on fold  0 is 0.714
Accuracy for   15 trees with depth 11 on fold  0 is 0.723
Accuracy for   16 trees with de

...

Accuracy for   43 trees with depth 11 on fold  8 is 0.728
Accuracy for   43 trees with depth 12 on fold  8 is 0.742
Accuracy for   43 trees with depth 13 on fold  8 is 0.734
Accuracy for   43 trees with depth 14 on fold  8 is 0.750
Optimal tree depth: 9
Number descent: 4
Accuracy for    1 trees with depth  9 on fold  0 is 0.616
Accuracy for    2 trees with depth  9 on fold  0 is 0.596
Accuracy for    3 trees with depth  9 on fold  0 is 0.678
Accuracy for    4 trees with depth  9 on fold  0 is 0.638
Accuracy for    5 trees with depth  9 on fold  0 is 0.688
Accuracy for    6 trees with depth  9 on fold  0 is 0.660
Accuracy for    7 trees with depth  9 on fold  0 is 0.688
Accuracy for    8 trees with depth  9 on fold  0 is 0.704
Accuracy for    9 trees with depth  9 on fold  0 is 0.711
Accuracy for   10 trees with depth  9 on fold  0 is 0.720
Accuracy for   11 trees with depth  9 on fold  0 is 0.716
Accuracy for   12 trees with depth  9 on fold  0 is 0.699
Accuracy for   13 trees with dep

Accuracy for   40 trees with depth  9 on fold  2 is 0.730
Accuracy for   41 trees with depth  9 on fold  2 is 0.723
Accuracy for   42 trees with depth  9 on fold  2 is 0.744
Accuracy for   43 trees with depth  9 on fold  2 is 0.735
Accuracy for   44 trees with depth  9 on fold  2 is 0.741
Accuracy for   45 trees with depth  9 on fold  2 is 0.724
Accuracy for   46 trees with depth  9 on fold  2 is 0.739
Accuracy for   47 trees with depth  9 on fold  2 is 0.730
Accuracy for   48 trees with depth  9 on fold  2 is 0.739
Accuracy for   49 trees with depth  9 on fold  2 is 0.725
Accuracy for    1 trees with depth  9 on fold  3 is 0.644
Accuracy for    2 trees with depth  9 on fold  3 is 0.630
Accuracy for    3 trees with depth  9 on fold  3 is 0.677
Accuracy for    4 trees with depth  9 on fold  3 is 0.670
Accuracy for    5 trees with depth  9 on fold  3 is 0.699
Accuracy for    6 trees with depth  9 on fold  3 is 0.689
Accuracy for    7 trees with depth  9 on fold  3 is 0.700
Accuracy for  

Accuracy for   35 trees with depth  9 on fold  5 is 0.736
Accuracy for   36 trees with depth  9 on fold  5 is 0.739
Accuracy for   37 trees with depth  9 on fold  5 is 0.752
Accuracy for   38 trees with depth  9 on fold  5 is 0.747
Accuracy for   39 trees with depth  9 on fold  5 is 0.742
Accuracy for   40 trees with depth  9 on fold  5 is 0.750
Accuracy for   41 trees with depth  9 on fold  5 is 0.741
Accuracy for   42 trees with depth  9 on fold  5 is 0.746
Accuracy for   43 trees with depth  9 on fold  5 is 0.751
Accuracy for   44 trees with depth  9 on fold  5 is 0.742
Accuracy for   45 trees with depth  9 on fold  5 is 0.755
Accuracy for   46 trees with depth  9 on fold  5 is 0.749
Accuracy for   47 trees with depth  9 on fold  5 is 0.737
Accuracy for   48 trees with depth  9 on fold  5 is 0.737
Accuracy for   49 trees with depth  9 on fold  5 is 0.738
Accuracy for    1 trees with depth  9 on fold  6 is 0.612
Accuracy for    2 trees with depth  9 on fold  6 is 0.632
Accuracy for  

Accuracy for   30 trees with depth  9 on fold  8 is 0.728
Accuracy for   31 trees with depth  9 on fold  8 is 0.720
Accuracy for   32 trees with depth  9 on fold  8 is 0.736
Accuracy for   33 trees with depth  9 on fold  8 is 0.719
Accuracy for   34 trees with depth  9 on fold  8 is 0.728
Accuracy for   35 trees with depth  9 on fold  8 is 0.734
Accuracy for   36 trees with depth  9 on fold  8 is 0.730
Accuracy for   37 trees with depth  9 on fold  8 is 0.736
Accuracy for   38 trees with depth  9 on fold  8 is 0.740
Accuracy for   39 trees with depth  9 on fold  8 is 0.731
Accuracy for   40 trees with depth  9 on fold  8 is 0.734
Accuracy for   41 trees with depth  9 on fold  8 is 0.734
Accuracy for   42 trees with depth  9 on fold  8 is 0.762
Accuracy for   43 trees with depth  9 on fold  8 is 0.725
Accuracy for   44 trees with depth  9 on fold  8 is 0.737
Accuracy for   45 trees with depth  9 on fold  8 is 0.735
Accuracy for   46 trees with depth  9 on fold  8 is 0.732
Accuracy for  

Accuracy for   43 trees with depth 10 on fold  8 is 0.743
Accuracy for   43 trees with depth 11 on fold  8 is 0.740
Accuracy for   43 trees with depth 12 on fold  8 is 0.738
Accuracy for   43 trees with depth 13 on fold  8 is 0.734
Accuracy for   43 trees with depth 14 on fold  8 is 0.715
Optimal tree depth: 11


### Test

In [8]:
X_test = df_test.loc[:, df_test.columns != 'Decision']

y_train = df['Decision']
X_train = df.loc[:, df.columns != 'Decision']

rfc = make_rfc(opt_num_trees, opt_tree_depth)
rfc.fit(X_train, y_train)

results = rfc.predict(X_test)
print("Runtime is {:} seconds".format(t.time() - start_time))

Runtime is 3283.734717130661 seconds


### Format for Output

In [9]:
df_test['Decision'] = results
df_test = df_test[['id', 'Decision']]
print(df_test)

df_test.to_csv(r"../data/random_forest_results.csv", index = False)

        id  Decision
0        1         1
1        2         0
2        3         0
3        4         1
4        5         1
...    ...       ...
2495  2496         1
2496  2497         1
2497  2498         1
2498  2499         1
2499  2500         0

[2390 rows x 2 columns]
